# Trie

In [3]:
class Trie:
    class Node:
        def __init__(self):
            self.val = None
            self.next = {}
            
        def __str__(self):
            out = str(self.val) + ':' + str(self.next.keys()) 
            return out
            
        def _print(self, depth):
            """Helper function only. Vizualization."""
            def _fill(k):
                return '│'*(k-1) + '└' if k>0 else ''
            out = ''
            first = True
            if (self.val is not None) and self.next: # Not a word, but a branching point
                out = out + '\n' + _fill(depth)
            for key,node in self.next.items():
                if first:
                    first = False
                else:
                    out = out + '\n' +  _fill(depth)
                out = out + key + node._print(depth+1)
            return out
            
        def _add(self, key, val):
            if not key: # We reached a target
                self.val = val
                return None
            ch = key[0]
            if ch not in self.next:
                self.next[ch] = Trie.Node()
            self.next[ch]._add(key[1:], val)
            
        def _collect(self, key=None):
            """Collect keys in a subtree. 
            KEY argument carries the accumulated key through recursion."""
            if key is None: key = ''
            if self.val is not None:
                out = [key]
            else:
                out = []
            if self.next:
                for ch,node in self.next.items():
                    out = out + node._collect(key+ch)
            return out
        
        def _size(self):
            """Returns a tuple: number of nodes, and number of keys."""
            if not next: return (1,1)
            else: 
                nnodes = 0
                nkeys = 0
                for key,node in self.next.items():
                    temp = node._size()
                    nnodes += temp[0]
                    nkeys += temp[1]
                if self.val is not None:
                    nkeys += 1
                return (1+nnodes, nkeys)
            
        def _find(self, key):
            if not key: return self.val            
            if key[0]=='?': # Wildcart
                out = [node._find(key[1:]) for _,node in self.next.items()]
                return [val for val in out if val is not None]
            if key[0] not in self.next: return None
            return self.next[key[0]]._find(key[1:])
        
        def _delete(self, key):
            if not key:             # Node to be deleted
                if not self.next:   # Leaf; cease to exist through returning None
                    return None
                self.val = None      # In-between node
                return self          # Set to "transit node", but don't cease to exist
            else:
                if key[0] not in self.next: 
                    raise KeyError(f'Trying to delete a non-existing key: {key}')
                    # Actually by that time it's not a full key, but only its latter part.
                    # So technically we'd probably had to send a message up, 
                    # and raise it at root level.
                branch = self.next[key[0]]._delete(key[1:])
                if branch is None:
                    del self.next[key[0]]
                    if not self.next and self.val is None:
                        return None
                else:
                    self.next[key[0]] = branch
                return self
            
    def __init__(self):
        self.root = Trie.Node()
        
    def __str__(self):
        return self.root._print(0)
        
    def add(self, key, val=None):
        if not key: return None
        self.root._add(key.lower(), val)
        
    def items(self):
        return self.root._collect()
    
    def size(self):
        """Returns a tuple: number of nodes, and number of keys."""
        return self.root._size()
    
    def find(self, key):
        return self.root._find(key)
    
    def delete(self, key):
        self.root._delete(key)
    
# test
t = Trie()
s = 'shall she sell sea shell at a shallow sea shore'
for token in s.split(' '):
    t.add(token, 100+len(token))
print(t.root)
print(t)
print(t.items())
print(t.size())
print(t.find('shell'))
print(t.find('shall'))
print(t.find('cat'))
print('sh?ll:',t.find('sh?ll'))
t.delete('shallow')
print(t)
t.delete('she')
print(t)
t.delete('horse')

None:dict_keys(['s', 'a'])
shall
││││└ow
│└e
││└ll
│└ore
└ell
│└a
a
└t
['shall', 'shallow', 'she', 'shell', 'shore', 'sell', 'sea', 'a', 'at']
(20, 9)
105
105
None
sh?ll: [105, 105]
shall
│└e
││└ll
│└ore
└ell
│└a
a
└t
shall
│└ell
│└ore
└ell
│└a
a
└t


KeyError: 'Trying to delete a non-existing key: horse'